## Imports

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (SystemMessagePromptTemplate, HumanMessagePromptTemplate, 
                                    PromptTemplate, ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.runnables import chain

import os

## Loading Environment Variables

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Selecting Chat Model

In [3]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_tokens=2000,
                        temperature=0.3)

## Setting Up Templates

In [4]:
#review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

In [5]:
prompt = """Given the user review below, classify it as either being about `Positive` or `Negative`.
            Do not respond with more than one word.

            Review: {review}
            Classification:"""

template = ChatPromptTemplate.from_template(template=prompt)
myChain = template | chat_model | StrOutputParser()


In [6]:
output = myChain.invoke({'review': review})
print(output)

Negative


In [7]:
# route positive responses one way and negative to another.
def route(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [8]:
# Positive
positive_prompt = """
                You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Review: {review}
                Answer:"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | chat_model | StrOutputParser()

# Negative
negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'udemy@kgptalkie.com'.
                Review: {review}
                Answer:"""


negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | chat_model | StrOutputParser()

## Setting up and Running the Chain

The "review" input is passed into prompt template, which is passed into chat_model, which is passed to StrOutputParser(), which is then passed into the function "route".

In [9]:
# Runnable Lambda - similar to lambda function ... runnableLambda is a pass through function
full_chain = {"sentiment": myChain, 'review': lambda x:x['review']} | RunnableLambda(route)

output = full_chain.invoke({'review': review })
print(output)

Dear [User's Name],

Thank you for taking the time to share your feedback. We sincerely apologize for the inconvenience you experienced with our service. Your satisfaction is important to us, and we regret that we did not meet your expectations.

We would appreciate the opportunity to address your concerns directly. Please feel free to reach out to us at udemy@kgptalkie.com, and we will do our best to resolve the issue and improve your experience.

Thank you for your understanding, and we hope to hear from you soon.

Best regards,  
[Your Name]  
[Your Position]  
KGPTalkie Team


## An Example of Sequential Chain using RunnableLambda

In [11]:
# Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda
# This is useful for formatting or when you need functionality not provided by other LangChain components,
# and custom functions used as Runnables are called RunnableLambdas.

def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

prompt = ChatPromptTemplate.from_template("Explain these inputs in 5 sentences: {input1} and {input2}")
myChain = prompt | chat_model | StrOutputParser() | {'char_counts': RunnableLambda(char_counts),
                                            'word_counts': RunnableLambda(word_counts),
                                            'output': RunnablePassthrough()}

# myChain = prompt | llm | StrOutputParser()
result = myChain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})
print(result)

{'char_counts': 665, 'word_counts': 111, 'output': "Earth is the third planet from the Sun in our solar system and is known for its ability to support life due to its atmosphere, liquid water, and suitable temperatures. It has a diverse range of ecosystems and climates, making it unique among the planets. The Sun, on the other hand, is a massive star at the center of our solar system, providing the necessary light and heat that sustain life on Earth. Composed primarily of hydrogen and helium, the Sun generates energy through nuclear fusion, which produces sunlight and warmth. Together, Earth and the Sun are part of a complex gravitational relationship that influences the planet's orbit, climate, and seasons."}
